## Simple Refelction Architecture

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage

from langchain.chat_models import ChatOpenAI

from langgraph.graph import END, MessageGraph

from typing import List, Sequence

In [14]:
class GenerationOutput(BaseModel):
    improved_prompt: str = Field(..., description="The improved prompt")


class ReflectionOutput(BaseModel):
    feedback: str = Field(..., description="The feedback goes here")

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.5,
)

async def generation_node(state: Sequence[BaseMessage]) -> List[BaseMessage]:
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are an AI engineer tasked with writing and improving prompts for a language model."
                " Generate the best possible prompt from the user request. Provide a detailed prompt that is clear and concise."
                " If the user provides critique, respond with a revised version of your previous attempts.",
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    generate = prompt | llm
    return await generate.ainvoke({"messages": state})

async def reflection_node(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
    # Other messages we need to adjust
    cls_map = {"ai": HumanMessage, "human": AIMessage}
    # First message is the original user request. We hold it the same for all nodes
    translated = [messages[0]] + [
        cls_map[msg.type](content=msg.content) for msg in messages[1:]
    ]
    reflection_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are an expert prompt engineer. Generate critique and recommendations for the user's prompt."
                " Provide detailed recommendations, including requests for context, examples, or clarifications.",
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    reflect = reflection_prompt | llm
    res = await reflect.ainvoke({"messages": translated})
    # We treat the output of this as human feedback for the generator
    return HumanMessage(content=res.content)

builder = MessageGraph()
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.set_entry_point("generate")

def should_continue(state: List[BaseMessage]):
    if len(state) > 6:
        # End after 3 iterations
        return END
    return "reflect"

builder.add_conditional_edges("generate", should_continue)
builder.add_edge("reflect", "generate")
graph = builder.compile()

In [15]:
base_prompt = "Populate the following function given its signature and docstring: {function_details}"

async for event in graph.astream(
    [
        HumanMessage(
            content=f"""Improve the following prompt: 
            
            Prompt: {base_prompt}"""
        )
    ],
):
    print(event)
    print("---")

{'generate': AIMessage(content='Certainly! Here’s an improved version of the prompt:\n\n---\n\n**Task:** Implement the function based on its provided signature and docstring.\n\n**Details:**\n- **Function Signature:** `{function_details}`\n- **Docstring:** `{docstring}`\n\n**Instructions:**\n1. Carefully read the function signature to understand the expected inputs and outputs.\n2. Review the docstring to comprehend the function\'s purpose, behavior, and any specific requirements or constraints.\n3. Write the complete function code that adheres to the given signature and fulfills the described functionality in the docstring.\n\n**Example:**\n- **Function Signature:** `def add_numbers(a: int, b: int) -> int:`\n- **Docstring:** `"""\n    Adds two integers and returns the result.\n    \n    Args:\n        a (int): The first integer.\n        b (int): The second integer.\n    \n    Returns:\n        int: The sum of the two integers.\n    """`\n\n**Implementation:**\n```python\ndef add_numb

In [16]:
print(event['generate'].content)

Thank you for the thoughtful feedback. Here’s the final version of the prompt, incorporating all the suggestions for maximum clarity and effectiveness:

---

**Task:** Implement the function based on its provided signature and docstring.

**Context:**
You are tasked with writing a function that adheres to a specific signature and fulfills the requirements described in its docstring. This function will be used as part of a larger codebase, so it is important to ensure it is both correct and efficient.

**Details:**
- **Function Signature:** `{function_details}`
- **Docstring:** `{docstring}`

**Instructions:**
1. **Understand the Function Signature:**
   - Review the function signature to identify the expected inputs and outputs, including their types.

2. **Analyze the Docstring:**
   - Carefully read the docstring to understand the function's purpose, behavior, and any specific requirements or constraints.
   - Pay attention to the description of the parameters and the return value.

